## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [9]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.tipo = tipo
        self._precio = self.chekear_precio(precio)  
        self._cantidad = self.chekear_cantidad(cantidad)  

    def valorTotal(self):
        return self._precio * self._cantidad

    def chekear_precio(self, precio):
        if 10 <= precio <= 10000:
            return precio
        else:
            return self._precio

    def chekear_cantidad(self, cantidad):   
        if 0 <= cantidad <= 10000:
            return cantidad
        else:
            return self._cantidad

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, nuevo_precio):
        self._precio = self.chekear_precio(nuevo_precio)

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, nueva_cantidad):
        self._cantidad = self.chekear_cantidad(nueva_cantidad)


In [10]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [11]:
class Oferta:
    def __init__(self, descripcion, codigos=[], tipos=[]):
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos = tipos

    def esAplicable(self, producto, cantidad):
        if producto.codigo in self.codigos or producto.tipo in self.tipos:
            return True
        else:
            return False

class OfertaDescuento(Oferta):
    def __init__(self, descuento, descripcion="Descuento 10%", *args, **kwargs):
        super().__init__(descripcion, *args, **kwargs)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            descuento_total = (self.descuento / 100) * producto.precio * cantidad
            return descuento_total
        else:
            return 0

class Oferta2x1(Oferta):
    def __init__(self, descripcion="Oferta 2x1", *args, **kwargs):
        super().__init__(descripcion, *args, **kwargs)

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad) and cantidad >= 2:
            cantidad_a_pagar = cantidad // 2
            return cantidad_a_pagar * producto.precio
        return 0


In [12]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo', 2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [13]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, 'r') as archivo_leer:
            catg_dic = csv.DictReader(archivo_leer)
            for fila in catg_dic:
                producto = Producto(fila['codigo'], fila['nombre'], float(fila['precio']), fila['tipo'], int(fila['cantidad']))
                catalogo.agregar(producto)
        return catalogo

    def guardar(self, archivo):
        with open(archivo, 'w', newline="") as archivo_guard:
            catg_lista = csv.writer(archivo_guard)
            catg_lista.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos:
                catg_lista.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])

    def agregar(self, *productos):
        if len(productos) == 1 and isinstance(productos[0], Producto):
            self.productos.append(productos[0])
        else:
            for producto in productos:
                self.productos.append(producto)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscarOferta(self, producto, cantidad):
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                return oferta
        return None

    def calcularDescuento(self, producto, cantidad):
        oferta = self.buscarOferta(producto, cantidad)
        if oferta:
            return oferta.calcularDescuento(producto, cantidad)
        return 0

    def vender(self, producto, cantidad):
        prod = self.buscar(producto.codigo)
        if prod:
            if prod.cantidad >= cantidad:
                prod.cantidad -= cantidad
            else:
                raise ValueError(f"No hay suficiente stock de {producto.nombre}")
        else:
            raise ValueError(f"Producto {producto.codigo} no encontrado en el catálogo")

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos)

    @property
    def valorTotal(self):
        return sum(producto.precio * producto.cantidad for producto in self.productos)

    @property
    def precioPromedio(self):
        if self.cantidadProductos == 0:
            return 0
        return self.valorTotal / self.cantidadProductos

    def informe(self):
        informe = []
        ahorro_descuento = 0
        ahorro_2x1 = 0

        informe.append(f"Cantidad de productos: {self.cantidadProductos}")
        informe.append(f"Cantidad de unidades: {self.cantidadUnidades}")
        informe.append(f"Precio Promedio: {self.precioPromedio:.2f}")
        informe.append(f"Valor total: {self.valorTotal:.2f}")

        tipos = set(producto.tipo for producto in self.productos)
        informe.append(f"Tipos de productos: \n                  • {'\n                  • '.join(tipos)}")

        for producto in self.productos:
            oferta = self.buscarOferta(producto, producto.cantidad)

            if oferta:
                descuento_producto = self.calcularDescuento(producto, producto.cantidad)

                if oferta.descripcion == "Oferta 2x1":
                    unidades_pagar = (producto.cantidad // 2) + (producto.cantidad % 2)
                    ahorro_2x1 += (producto.cantidad - unidades_pagar) * producto.precio

                elif oferta.descripcion == "Descuento 10%":
                    ahorro_descuento += descuento_producto

        if self.ofertas:
            informe.append(f"Ofertas: {ahorro_descuento:.2f}")
            informe.append(f"Oferta 2x1: {ahorro_2x1:.2f}")

        return '\n'.join(informe)


In [14]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [15]:
import re
class Cliente:
    def __init__(self, nombre, cuit):
        # Se inicializan los atributos con valores proporcionados, a través de los setters
        self.nombre = nombre
        self.cuit = cuit

    # Getter y setter para el nombre
    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
        if value.strip() == "":  # Si el nombre está vacío o tiene solo espacios
            print("Error: El nombre no puede estar vacío.")
        else:
            self._nombre = value

    # Getter y setter para el CUIT
    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, value):
        # Validación del CUIT con expresión regular
        if re.match(r'^\d{2}-\d{8}-\d$', value):
            self._cuit = value
        else:
            print("Error: El CUIT debe tener el formato XX-XXXXXXXX-X y debe contener 11 dígitos.")


In [16]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Error: El nombre no puede estar vacío.
Error: El CUIT debe tener el formato XX-XXXXXXXX-X y debe contener 11 dígitos.
Error: El CUIT debe tener el formato XX-XXXXXXXX-X y debe contener 11 dígitos.
Prueba pasada exitosamente!


In [17]:
import datetime

class Factura:
    _ultimo_numero = 100

    @classmethod
    def ultimaFactura(cls, numero):
        cls._ultimo_numero = numero

    @classmethod
    def nuevoNumero(cls):
        cls._ultimo_numero += 1
        return cls._ultimo_numero

    def __init__(self, catalogo, cliente):
        self.numero = Factura.nuevoNumero()
        self.fecha = datetime.datetime.now().strftime("%d/%m/%Y")
        self.cliente = cliente
        self.catalogo = catalogo
        self.items = []  
        self.subtotal = 0
        self.descuentos = 0

    @property
    def cantidadProductos(self):
        return len(self.items)

    @property
    def cantidadUnidades(self):
        return sum(cantidad for _, cantidad in self.items)

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def agregar(self, producto, cantidad):
        # Vende el producto a través del catálogo
        self.catalogo.vender(producto, cantidad)

        # Actualiza el item en la lista de items de la factura
        for i, (prod, cant) in enumerate(self.items):
            if prod.codigo == producto.codigo:
                self.items[i] = (prod, cant + cantidad)
                break
        else:
            self.items.append((producto, cantidad))

        self.actualizarTotales()

    def actualizarTotales(self):
        self.subtotal = 0
        self.descuentos = 0
        for producto, cantidad in self.items:
            subtotal_producto = producto.precio * cantidad
            self.subtotal += subtotal_producto

            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            self.descuentos += descuento

    def imprimir(self):
        factura = []
        factura.append(f"Factura: {self.numero}")
        factura.append(f"Fecha  : {self.fecha}")
        factura.append(f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})")
        factura.append("")

        for producto, cantidad in self.items:
            subtotal_producto = producto.precio * cantidad
            factura.append(f"- {cantidad} {producto.nombre:<25} x ${producto.precio:.2f} = ${subtotal_producto:.2f}")

            oferta = self.catalogo.buscarOferta(producto, cantidad)
            if oferta:
                descuento_producto = self.catalogo.calcularDescuento(producto, cantidad)
                factura.append(f"      {oferta.descripcion:<35} - ${descuento_producto:.2f}")

        factura.append("")
        factura.append(f"{'Subtotal:':>50}   ${self.subtotal:.2f}")
        factura.append(f"{'Descuentos:':>50} - ${self.descuentos:.2f}")
        factura.append(f"{'-' * 50}")
        factura.append(f"{'TOTAL:':>50}   ${self.total:.2f}")

        return '\n'.join(factura)


In [18]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [19]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
